<a href="https://colab.research.google.com/github/xprilion/gemini-function-calling-workshop/blob/main/GeminiFunctionCallingWithRAGChromaDb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Putting together Function Calling and RAGs with Gemini

In this notebook, we explore using Gemini APIs with Function Calling and Retrieval Augmented Generation. It covers a method to use ChromaDB powered vector store to act as a cache for an external API.

We use the [Frankfurter API](https://www.frankfurter.app/) for fetching exchange rates between currencies.

## Setup

1. Install ChromaDB
2. Install Google's SDK for Gen AI

This notebook uses Google AI Studio for calling the APIs. This method is slightly different from using Vertex AI Studio based Gemini Model APIs.

In [1]:
%%capture
!pip install chromadb
!pip install -U -q google-generativeai

In [2]:
import requests
import google.generativeai as genai
import chromadb
from datetime import datetime, timedelta

Instantiating a Gemini API client.

In [3]:
try:
    # Used to securely store your API key
    from google.colab import userdata

    # Or use `os.getenv('API_KEY')` to fetch an environment variable.
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
except ImportError:
    import os
    GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

genai.configure(api_key=GOOGLE_API_KEY)

Instantiating a ChromaDB client

In [4]:
chroma_client = chromadb.Client()

Creating a collection, you can have multiple collections per client. These are all stored in memory, so make sure your RAM is high enough to hold these documents.

In [5]:
collection = chroma_client.create_collection(name="exchange-rates")

## [Optional] Initialize the collection with latest 5 days of data on exchange rates.

Let's get the latest 5 dates for which we'll fetch the initial chunk of data into the DB.

In [6]:
current_date = datetime.now()
dates = [(current_date - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(1, 6)]

In [7]:
dates

['2024-05-10', '2024-05-09', '2024-05-08', '2024-05-07', '2024-05-06']

In [8]:
url = "https://api.frankfurter.app/latest"
response = requests.get(url)
api_data = response.json()

currencies = list(api_data['rates'].keys())

In [9]:
def initial_fetch_conversion_rates(date, currencies):
    rates = {}
    try:
        response = requests.get(f'https://api.frankfurter.app/{date}')
        data = response.json()
        rates = {curr: data['rates'].get(curr, 'N/A') for curr in currencies}
    except Exception as e:
        print(f"Error fetching conversion rates for {date}: {e}")
    return rates

In [10]:
documents = []
metadatas = []
ids = []
for date in dates:
    rates = initial_fetch_conversion_rates(date, currencies)
    for currency, rate in rates.items():
        document = f"EUR to {currency} conversion rate is {rate} on {date}"
        documents.append(document)
        metadata = {"base": "EUR", "target": currency, "date": date, "rate": rate}
        id_ = f"EUR-{currency}-{date.replace('-', '')}"
        metadatas.append(metadata)
        ids.append(id_)

In [11]:
collection.add(
    metadatas=metadatas,
    documents=documents,
    ids=ids,
)

Quick check of whether our initial fetch of data works!

In [12]:
search_query = "EUR to INR conversion rate on 09-05-2024"

In [13]:
results = collection.query(
    query_texts=[search_query], # Chroma will embed this for you
    n_results=1 # how many results to return
)
results

{'ids': [['EUR-INR-20240509']],
 'distances': [[0.08850838989019394]],
 'metadatas': [[{'base': 'EUR',
    'date': '2024-05-09',
    'rate': 89.61,
    'target': 'INR'}]],
 'embeddings': None,
 'documents': [['EUR to INR conversion rate is 89.61 on 2024-05-09']],
 'uris': None,
 'data': None}

## Function for fetching data by date, from and target currency directly from API

We'll call this API anytime we need to fetch new data from the API.

In [14]:
def get_exchange_rate_from_api(currency_from, currency_to, currency_date="latest"):
    try:
        response = requests.get(f'https://api.frankfurter.app/{currency_date}', params={
            "from": currency_from,
            "to": currency_to
        })
        metadata, document, id_ = None, None, None
        data = response.json()

        rate = data['rates'][currency_to]
        document = f"{currency_from} to {currency_to} conversion rate is {rate} on {currency_date}"
        metadata = {"base": currency_from, "target": currency_to, "date": currency_date, "rate": rate}
        id_ = f"{currency_from}-{currency_to}-{currency_date.replace('-', '')}"

        collection.upsert(
            metadatas=[metadata],
            documents=[document],
            ids=[id_],
        )

        print("Added document")
        print(document)
    except Exception as e:
        print(f"Error fetching conversion rates for {currency_date}: {e}")
        raise(e)
    return document

Quick check if our function above is working!

In [15]:
get_exchange_rate_from_api("USD", "INR", "2024-05-10")

Added document
USD to INR conversion rate is 83.51 on 2024-05-10


'USD to INR conversion rate is 83.51 on 2024-05-10'

## Function that is ready to work with Gemini API

This function first checks if we have the data for those dates and currrency pair already in the database. If not, fetch from API and return.

Check out all those verbose descriptions, these help Gemini understand this function better while deciding when and how to call it!

In [47]:
def get_exchange_rate(currency_from: str, currency_to: str, currency_date: str ="latest"):
    """
    This function retrieves the exchange rate between two currencies on a specific date.

    Args:
        currency_from (str): The currency to convert from (ISO 4217 format). (Required)
        currency_to (str): The currency to convert to (ISO 4217 format). (Required)
        currency_date (str, optional): The date for the exchange rate in YYYY-MM-DD format
                                        or "latest" for the most recent rate. Defaults to "latest".

    Returns:
        float: The exchange rate (currency_to per unit of currency_from).
            If the rate cannot be retrieved, returns None.

    Raises:
        ValueError: If either currency code is invalid or the date format is incorrect.
    """

    ## First, check if the Vector Store has the data
    search_query = f"{currency_from} to {currency_to} conversion on {currency_date}"

    results = collection.query(
        query_texts=[search_query], # Chroma will embed this for you
        n_results=1 # how many results to return
    )

    if (results["ids"][0][0] == f"{currency_from}-{currency_to}-{currency_date.replace('-', '')}"):
        print("Vector Store hit, let's return this.")
        print("DB return: ", results["documents"][0])
        return results["documents"][0]

    api_data = get_exchange_rate_from_api(currency_from, currency_to, currency_date)
    print("Vector store miss. Fetching data from API and returning!")
    print("API return: ",api_data )
    return api_data

Quick check of above function running!

In [48]:
get_exchange_rate("USD", "INR", "2024-05-01")

Vector Store hit, let's return this.
DB return:  ['USD to INR conversion rate is 83.43 on 2024-05-01']


['USD to INR conversion rate is 83.43 on 2024-05-01']

## Declare the model with the function declaration

The SDK converts your function to a declaration for usage, if does not send the function anywhere!

In [49]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              tools=[get_exchange_rate])

Turns out chat usage has automatic function calling enabled, let's use it for the oomph factor!

In [50]:
chat = model.start_chat(enable_automatic_function_calling=True)

Wrap the chat invocation in a nice neat function!

In [51]:
def ask_gemini(query):
  response = chat.send_message(query)
  return response.text

## Testing

Check: 20 USD to INR on 2024-05-10

In [52]:
ask_gemini('I have 20 US dollars with me. Calculate how much I have in INR as of 10th May, 2024.')

Vector Store hit, let's return this.
DB return:  ['USD to INR conversion rate is 83.51 on 2024-05-10']


'20 US dollars would be equal to 1,670.2 INR as of May 10th, 2024.'

Check: 20 USD to INR on 2024-05-01

In [53]:
ask_gemini('I have 20 US dollars with me. Calculate how much I have in INR as of 1st May, 2024?')

Vector Store hit, let's return this.
DB return:  ['USD to INR conversion rate is 83.43 on 2024-05-01']


'20 US dollars would be equal to 1,668.6 INR as of May 1st, 2024.'

Check: 20 USD to INR on 2024-01-05

In [54]:
ask_gemini('I have 20 US dollars with me. Calculate how much I have in INR as of 5th January, 2024?')

Added document
USD to INR conversion rate is 83.15 on 2024-01-05
Vector store miss. Fetching data from API and returning!
API return:  USD to INR conversion rate is 83.15 on 2024-01-05


'20 US dollars would be equal to 1,663 INR as of January 5th, 2024.'